# Initialization

In [1]:
from PIL import Image
import numpy as np
import deeplake
import os
import random
from sklearn.metrics import mean_squared_error
import warnings
import bitarray

C:\Users\NOSHIN\AppData\Roaming\Python\Python311\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.4.1) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
# Initializing cover & secret image row column and chromosome total bit count

cover_imageRow = 256
cover_imageCol = 256
secret_imageRow = 64
secret_imageCol = 128
chromosome_totalBitCount = 60

In [42]:
SECRET_IMAGE = "secret_002.jpg"

COVER_IMAGE_DIRECTORY    = "E:\\Projects\\Thesis\\Thesis_Implementation\\Dataset\\Unsplash_990_256_by_256"
SECRET_IMAGE_DIRECTORY   = "E:\\Projects\\Thesis\\Thesis_Implementation\\Dataset\\Secret_Images_64_by_128"
STEGO_IMAGE_DIRECTORY    = "E:\\Projects\\Thesis\\Thesis_Implementation\\Dataset\\Stego_Images"
DECODED_IMAGE_DIRECTORY  = "E:\\Projects\\Thesis\\Thesis_Implementation\\Dataset\\Decoded_Images"

In [43]:
# Change to dataset directory

os.chdir(COVER_IMAGE_DIRECTORY)
%pwd

'E:\\Projects\\Thesis\\Thesis_Implementation\\Dataset\\Unsplash_990_256_by_256'

In [32]:
# Converting image to numpy array

def imageToNumpyArray(image):
    im = Image.open(image)
    im1 = Image.Image.split(im)

    R_channel = np.array(im1[0])
    G_channel = np.array(im1[1])
    B_channel = np.array(im1[2])

    # Convert numpy array to binary numpy array

    R_channel = R_channel.astype('uint8')
    G_channel = G_channel.astype('uint8')
    B_channel = B_channel.astype('uint8')

    R_channelBinary = np.unpackbits(R_channel, axis=1)
    G_channelBinary = np.unpackbits(G_channel, axis=1)
    B_channelBinary = np.unpackbits(B_channel, axis=1)

   

    return R_channelBinary, G_channelBinary, B_channelBinary

In [8]:
# Counting 1 at the LSB and second LSB of each pixel

def count_oneCover(channelBinary, channel_row, channel_col):
   channelBinary = channelBinary.reshape(channel_row*channel_col, 8)
   last_two_bits = channelBinary[:, -2:]
    
   ones_count = np.count_nonzero(last_two_bits == 0b01) + np.count_nonzero(last_two_bits == 0b11)
   percentage = ones_count / (channel_row * channel_col * 2) * 100
    
   return percentage

In [9]:
# Counting 1 at all bits of each pixel

def count_one(channelBinary, channel_row, channel_col):
    count = 0
    for i in range(channelBinary.shape[0]):
        for j in range(channelBinary.shape[1]):
            if channelBinary[i][j] == 1:
                    count += 1
                    
    return count/(channel_row*channel_col*8)*100

In [44]:
# Preparing SECRET image

os.chdir(SECRET_IMAGE_DIRECTORY)

R_secretChannelBinary, G_secretChannelBinary, B_secretChannelBinary = imageToNumpyArray(SECRET_IMAGE)


R_secretLSB = count_one(R_secretChannelBinary, secret_imageRow, secret_imageCol)
G_secretLSB = count_one(G_secretChannelBinary, secret_imageRow, secret_imageCol)
B_secretLSB = count_one(B_secretChannelBinary, secret_imageRow, secret_imageCol)

print(R_secretLSB," ",G_secretLSB," ",B_secretLSB)

40.9393310546875   43.42803955078125   37.85858154296875


In [45]:
# Preparing weight list for cover images


os.chdir(COVER_IMAGE_DIRECTORY)

total_files = 0
for filename in os.listdir(COVER_IMAGE_DIRECTORY):
    total_files += 1

weight_list = [[] for i in range(total_files)]


# weight_track = 0
i=0
for filename in os.listdir(COVER_IMAGE_DIRECTORY):
    # if i==10:
    #     break
    
    R_channelBinary, G_channelBinary, B_channelBinary = imageToNumpyArray(filename)
    if i==0:
        channel_row = R_channelBinary.shape[0]
        channel_col = int(R_channelBinary.shape[1]/8)

    R_oneCount = count_oneCover(R_channelBinary, channel_row, channel_col)
    G_oneCount = count_oneCover(G_channelBinary, channel_row, channel_col)
    B_oneCount = count_oneCover(B_channelBinary, channel_row, channel_col)

    # print(R_oneCount, G_oneCount, B_oneCount,"\n")

    weight = abs(R_secretLSB + G_secretLSB + B_secretLSB - R_oneCount - G_oneCount - B_oneCount)
    weight_list[i].append(weight)
    weight_list[i].append(filename)

    print(filename, weight)

    i += 1

cover_000.jpg 24.483489990234375
cover_001.jpg 28.926849365234375
cover_002.jpg 27.20947265625
cover_003.jpg 23.9349365234375
cover_004.jpg 26.580047607421875
cover_005.jpg 20.314788818359375
cover_006.jpg 24.9237060546875
cover_007.jpg 26.335906982421875
cover_008.jpg 27.870941162109375
cover_009.jpg 34.7869873046875
cover_010.jpg 28.43780517578125
cover_011.jpg 31.1676025390625
cover_012.jpg 29.190826416015625
cover_013.jpg 27.7496337890625
cover_014.jpg 27.214813232421875
cover_015.jpg 22.85003662109375
cover_016.jpg 23.883056640625
cover_017.jpg 28.271484375
cover_018.jpg 22.394561767578125
cover_019.jpg 27.38037109375
cover_020.jpg 27.909088134765625
cover_021.jpg 27.0599365234375
cover_022.jpg 26.42364501953125
cover_023.jpg 27.97088623046875
cover_024.jpg 29.053497314453125
cover_025.jpg 12.070465087890625
cover_026.jpg 35.007476806640625
cover_027.jpg 28.076171875
cover_028.jpg 28.864288330078125
cover_029.jpg 28.794097900390625
cover_030.jpg 23.07891845703125
cover_031.jpg 27.

In [46]:
# Picking 10 cover images with minimum weight

cover_imageList = []*10
weight_list.sort()

for i in range(10):
    cover_imageList.append(weight_list[i][1])
    print(weight_list[i][1], weight_list[i][0])

cover_590.jpg 0.6683349609375
cover_402.jpg 2.008819580078125
cover_385.jpg 2.45819091796875
cover_986.jpg 2.620697021484375
cover_095.jpg 2.910614013671875
cover_405.jpg 3.908538818359375
cover_711.jpg 4.172515869140625
cover_597.jpg 4.6722412109375
cover_151.jpg 5.0048828125
cover_932.jpg 5.556488037109375


# Encoding Preparation

In [36]:
# Modified LCG algorithm

def modified_LCG2(seed, a, c, M):
    if a>0:
        sequence = []
        used_numbers = set()
        while len(sequence) < M:
            if seed in used_numbers:
                for i in range(seed + 1, M):
                    if i not in used_numbers:
                        seed = i
                        break
                else:
                    for i in range(0, seed):
                        if i not in used_numbers:
                            seed = i
                            break
            used_numbers.add(seed)
            sequence.append(seed)
            seed = (a*seed + c) % M
        return sequence
    else:
        sequence = list(range(M))
        used = set()
        x_n = seed
        for i in range(M):
            if x_n in used:
                x_n = next((i for i in range(x_n + 1, M) if i not in used), next((i for i in range(x_n) if i not in used), None))
            else:
                x_n = (a * x_n + c) % M
            used.add(x_n)
            sequence[i], sequence[x_n] = sequence[x_n], sequence[i]
        return sequence

In [47]:
# Fitness function
CONST = 255**2
CONST_LOG = 10 * np.log10(CONST)

def calculate_PSNR(mse):
    return 10 * np.log10(CONST / mse)



def calculate_ssim(r1, g1, b1, r2, g2, b2):
    # Calculate the constants
    c1 = (0.01 * 255) ** 2
    c2 = (0.03 * 255) ** 2
    c3 = c2 / 2
    
    # Calculate the means of each channel for both images
    mu_r1, mu_g1, mu_b1 = np.mean(r1), np.mean(g1), np.mean(b1)
    mu_r2, mu_g2, mu_b2 = np.mean(r2), np.mean(g2), np.mean(b2)
    
    # Calculate the variances of each channel for both images
    var_r1, var_g1, var_b1 = np.var(r1), np.var(g1), np.var(b1)
    var_r2, var_g2, var_b2 = np.var(r2), np.var(g2), np.var(b2)
    
    # Calculate the covariance between the channels of the two images
    cov_rg = np.cov(r1.flatten(), g1.flatten(), rowvar=False)[0][1]
    cov_rb = np.cov(r1.flatten(), b1.flatten(), rowvar=False)[0][1]
    cov_gb = np.cov(g1.flatten(), b1.flatten(), rowvar=False)[0][1]
    
    # Calculate the luminance, contrast, and structure components
    l_r = (2 * mu_r1 * mu_r2 + c1) / (mu_r1 ** 2 + mu_r2 ** 2 + c1)
    l_g = (2 * mu_g1 * mu_g2 + c1) / (mu_g1 ** 2 + mu_g2 ** 2 + c1)
    l_b = (2 * mu_b1 * mu_b2 + c1) / (mu_b1 ** 2 + mu_b2 ** 2 + c1)
    
    c_r = (2 * cov_rg + c2) / (var_r1 + var_r2 + c2)
    c_g = (2 * cov_gb + c2) / (var_g1 + var_g2 + c2)
    c_b = (2 * cov_rb + c2) / (var_b1 + var_b2 + c2)
    
    s_r = (cov_rg + c3) / (np.sqrt(var_r1) * np.sqrt(var_r2) + c3)
    s_g = (cov_gb + c3) / (np.sqrt(var_g1) * np.sqrt(var_g2) + c3)
    s_b = (cov_rb + c3) / (np.sqrt(var_b1) * np.sqrt(var_b2) + c3)
    
    # Calculate the overall SSIM index
    ssim_r = l_r * c_r * s_r
    ssim_g = l_g * c_g * s_g
    ssim_b = l_b * c_b * s_b
    ssim_total = (ssim_r + ssim_g + ssim_b) / 3.0

    # print("SSIM : ", ssim_r, ssim_g, ssim_b, ssim_total)
    
    return ssim_total



In [48]:
# Converting secret image RGB channels to binary 1D array

R_secretChannelBinary_1D = R_secretChannelBinary.flatten()
G_secretChannelBinary_1D = G_secretChannelBinary.flatten()
B_secretChannelBinary_1D = B_secretChannelBinary.flatten()


# Checking shape
print(R_secretChannelBinary_1D.shape)
print(G_secretChannelBinary_1D.shape)
print(B_secretChannelBinary_1D.shape)

(65536,)
(65536,)
(65536,)


In [16]:
# initialize population of 100 chromosomes


def create_population(population_length):
    population = []*population_length

    for i in range(population_length):
        chromosome = []

        
        # Randomly selecting channels of cover image for secret image channels

        # R_channelPosition = random.randint(0, 2)

        # if R_channelPosition == 0:
        #     L = [1,2]
        #     G_channelPosition = random.choice(L)
        # if R_channelPosition == 1:
        #     L = [0,2]
        #     G_channelPosition = random.choice(L)
        # if R_channelPosition == 2:
        #     L = [0,1]
        #     G_channelPosition = random.choice(L)

        # for j in range(3):
        #     if j != R_channelPosition and j != G_channelPosition:
        #         B_channelPosition = j
        #         break

        
        # Randomly selecting parameters for the chromosome

        number_of_partitions = random.randint(0, 7)
        multiplier_a = random.randint(0, 65535)
        Offset_c = random.randint(0, 65535)
        initial_seed = random.randint(0, int(cover_imageRow * cover_imageCol*2*3 / (2**number_of_partitions))-1)
        # initial_seed = random.randint(0, int(R_secretChannelBinary_1D.shape[0] / (2**number_of_partitions))-1)

        # chromosome.append(R_channelPosition)
        # chromosome.append(G_channelPosition)
        # chromosome.append(B_channelPosition)
        
        chromosome.append(number_of_partitions)
        chromosome.append(multiplier_a)
        chromosome.append(Offset_c)
        chromosome.append(initial_seed)

        population.append(chromosome)

        # print(population)

    return population


In [49]:
population = create_population(100)
print(len(population))
for i in range(100):
    print(i, population[i])

100
0 [0, 224, 11298, 78401]
1 [2, 55159, 28147, 60593]
2 [0, 16985, 20603, 25870]
3 [6, 55297, 15660, 748]
4 [1, 35277, 20132, 140846]
5 [1, 61329, 52991, 102269]
6 [7, 51220, 15545, 84]
7 [6, 20720, 1963, 4713]
8 [2, 33757, 15054, 71598]
9 [6, 22813, 14378, 1321]
10 [3, 47990, 1341, 19448]
11 [4, 27834, 39594, 22438]
12 [7, 5798, 61144, 1681]
13 [0, 16061, 16627, 158674]
14 [2, 14153, 27072, 11931]
15 [0, 57667, 28595, 133817]
16 [4, 43262, 33588, 3021]
17 [2, 59261, 47335, 55040]
18 [3, 44302, 44723, 29631]
19 [6, 58471, 50284, 2976]
20 [4, 56301, 44602, 7460]
21 [0, 45390, 34348, 143760]
22 [7, 35668, 28922, 1481]
23 [4, 38650, 4146, 23567]
24 [7, 65475, 35594, 2508]
25 [6, 18894, 9421, 2644]
26 [2, 18774, 21977, 13254]
27 [1, 54794, 51028, 121388]
28 [7, 27699, 20923, 2762]
29 [3, 57346, 51051, 13636]
30 [4, 49417, 60332, 262]
31 [0, 52902, 36517, 114952]
32 [0, 18951, 59760, 81227]
33 [1, 29144, 32626, 117337]
34 [4, 1049, 60729, 11473]
35 [1, 22522, 9288, 7411]
36 [4, 24823, 579

In [18]:
# converting chromosome into binary array

def chromosome_to_binary(chromosome):
    chromosome = np.asarray(chromosome)
    # R_channelPosition = np.binary_repr(chromosome[0], width=2)
    # G_channelPosition = np.binary_repr(chromosome[1], width=2)
    # B_channelPosition = np.binary_repr(chromosome[2], width=2)
    partition_bits = np.binary_repr(chromosome[0], width=3)
    multiplier_a = np.binary_repr(chromosome[1], width=16)
    Offset_c = np.binary_repr(chromosome[2], width=16)
    initial_seed = np.binary_repr(chromosome[3], width=19)
    # chromosome_binary = R_channelPosition + G_channelPosition + B_channelPosition + partition_bits + multiplier_a + Offset_c + initial_seed
    chromosome_binary = partition_bits + multiplier_a + Offset_c + initial_seed
    return chromosome_binary

In [19]:
# Extraction of last 2 bits of each pixel

def lastTwoBits_extraction(channelBinary, channel_row, channel_col):
    channelBinary = channelBinary.reshape(channel_row*channel_col, 8)
    last_two_bitsArray = channelBinary[:, -2:]

    return last_two_bitsArray

In [20]:
# Insert secret data channel into cover image channel

def insert_secret_data(cover_channel, secret_channel, rearranging_sequence, chromosome):
    secret_channelIndex = 0
    for i in range(len(rearranging_sequence)):
        if secret_channelIndex >= secret_channel.shape[0]:
            break
        for j in range((2**int(chromosome[0]))*rearranging_sequence[i], (2**int(chromosome[0]))*(rearranging_sequence[i]+1)):
            cover_channel[j] = secret_channel[secret_channelIndex]
            secret_channelIndex += 1

    return cover_channel

In [21]:
# Creating stego image


def create_stego_image(chromosome, cover_image, R_secretChannelBinary_1D, G_secretChannelBinary_1D, B_secretChannelBinary_1D):

    R_channelBinary, G_channelBinary, B_channelBinary = imageToNumpyArray(cover_image)
    rearranging_sequence = modified_LCG2(chromosome[3], chromosome[1], chromosome[2], int(cover_imageRow * cover_imageCol*2*3 / (2**chromosome[0])) )
    
    R_channelBinaryOriginal = lastTwoBits_extraction(R_channelBinary, cover_imageRow, cover_imageCol)
    G_channelBinaryOriginal = lastTwoBits_extraction(G_channelBinary, cover_imageRow, cover_imageCol)
    B_channelBinaryOriginal = lastTwoBits_extraction(B_channelBinary, cover_imageRow, cover_imageCol)

    R_channelBinary_flatten = R_channelBinaryOriginal.flatten()
    G_channelBinary_flatten = G_channelBinaryOriginal.flatten()
    B_channelBinary_flatten = B_channelBinaryOriginal.flatten()
    
    concatenated_array = np.concatenate((R_channelBinary_flatten, G_channelBinary_flatten, B_channelBinary_flatten), axis=0)
    secret_channel = np.concatenate((R_secretChannelBinary_1D, G_secretChannelBinary_1D, B_secretChannelBinary_1D), axis=0)

    temp_concatenatedArray = np.copy(concatenated_array)

    concatenated_arrayModified = insert_secret_data(temp_concatenatedArray, secret_channel, rearranging_sequence, chromosome)

    # Splitting the concatenated array into 3 channels
    R_channelBinaryModified = concatenated_arrayModified[:cover_imageRow*cover_imageCol*2]
    G_channelBinaryModified = concatenated_arrayModified[cover_imageRow*cover_imageCol*2:cover_imageRow*cover_imageCol*4]   
    B_channelBinaryModified = concatenated_arrayModified[cover_imageRow*cover_imageCol*4:]

    # Creating 3 stego channels by inserting channelBinaryModified into channelBinary
    R_stegoChannelBinary = np.copy(R_channelBinary)
    G_stegoChannelBinary = np.copy(G_channelBinary)
    B_stegoChannelBinary = np.copy(B_channelBinary)

    
    count = 0
    for i in range(R_channelBinary.shape[0]):
        for j in range(R_channelBinary.shape[1]):
            # print(count)
            if (j+1)%8 == 0 or (j+2)%8==0:
                R_stegoChannelBinary[i][j] = R_channelBinaryModified[count]
                count += 1
    count = 0
    for i in range(G_channelBinary.shape[0]):
        for j in range(G_channelBinary.shape[1]):
            if (j+1)%8 == 0 or (j+2)%8==0:
                G_stegoChannelBinary[i][j] = G_channelBinaryModified[count]
                count += 1
    count = 0
    for i in range(B_channelBinary.shape[0]):
        for j in range(B_channelBinary.shape[1]):
            if (j+1)%8 == 0 or (j+2)%8==0:
                B_stegoChannelBinary[i][j] = B_channelBinaryModified[count]
                count += 1

    # converting chromosome into binary array
    chromosome_binary = chromosome_to_binary(chromosome)
    # print("Binary chromosome ", len(chromosome_binary), chromosome_binary)

    # Embed chromosome into blue cover channel
    chromosome_bitCount = 0
    for i in range(B_channelBinary.shape[0]):
        for j in range(B_channelBinary.shape[1]):
            if (j+3)%8 == 0 and chromosome_bitCount < len(chromosome_binary):
                B_stegoChannelBinary[i][j] = int(chromosome_binary[chromosome_bitCount])
                chromosome_bitCount += 1

    return R_stegoChannelBinary, G_stegoChannelBinary, B_stegoChannelBinary

In [50]:
os.chdir(COVER_IMAGE_DIRECTORY)

print(os.getcwd())
print("C ",cover_imageList[1])

R_channelBinary, G_channelBinary, B_channelBinary = imageToNumpyArray(cover_imageList[1])

ch = [5, 14, 43236, 12000]
print(population[0])
R_stegoChannelBinary, G_stegoChannelBinary, B_stegoChannelBinary= create_stego_image(population[0], cover_imageList[1], R_secretChannelBinary_1D, G_secretChannelBinary_1D, B_secretChannelBinary_1D)
mse = mean_squared_error(R_channelBinary, R_stegoChannelBinary) + mean_squared_error(G_channelBinary, G_stegoChannelBinary) + mean_squared_error(B_channelBinary, B_stegoChannelBinary)
print("MSE : ", mse)
PSNR = calculate_PSNR(mse)
print("PSNR : ", PSNR)

E:\Projects\Thesis\Thesis_Implementation\Dataset\Unsplash_990_256_by_256
C  cover_402.jpg
[0, 224, 11298, 78401]
MSE :  0.18176841735839844
PSNR :  55.53561934999061


In [23]:
# TESTING

os.chdir(COVER_IMAGE_DIRECTORY)

best_score = 0
best_coverImage = ""

for i in range(len(cover_imageList)):
    R_channelBinary, G_channelBinary, B_channelBinary = imageToNumpyArray(cover_imageList[i])
    R_stegoChannelBinary, G_stegoChannelBinary, B_stegoChannelBinary = create_stego_image(ch, cover_imageList[i], R_secretChannelBinary_1D, G_secretChannelBinary_1D, B_secretChannelBinary_1D)
    mse = mean_squared_error(R_channelBinary, R_stegoChannelBinary) + mean_squared_error(G_channelBinary, G_stegoChannelBinary) + mean_squared_error(B_channelBinary, B_stegoChannelBinary)
    PSNR = calculate_PSNR(mse)
    print("Cover image : ",cover_imageList[i][1]," ",cover_imageList[i][0])
    print("Mean Square Error : ",mse)
    print("PSNR : ",PSNR)
    if PSNR > best_score:
        best_score = PSNR
        best_coverImage = cover_imageList[i]

print("Best score : ",best_score)
print("Best cover image : ",best_coverImage)

Cover image :  o   c
Mean Square Error :  0.1882171630859375
PSNR :  55.38421137665854
Cover image :  o   c
Mean Square Error :  0.18779373168945312
PSNR :  55.39399268880918
Cover image :  o   c
Mean Square Error :  0.1883697509765625
PSNR :  55.380691972236576
Cover image :  o   c
Mean Square Error :  0.18668174743652344
PSNR :  55.41978503417553
Cover image :  o   c
Mean Square Error :  0.18611526489257812
PSNR :  55.43298366099653
Cover image :  o   c
Mean Square Error :  0.18688201904296875
PSNR :  55.41512843363242
Cover image :  o   c
Mean Square Error :  0.1876201629638672
PSNR :  55.39800852019941
Cover image :  o   c
Mean Square Error :  0.18749046325683594
PSNR :  55.40101178725588
Cover image :  o   c
Mean Square Error :  0.18758201599121094
PSNR :  55.39889161847081
Cover image :  o   c
Mean Square Error :  0.1866741180419922
PSNR :  55.41996252724593
Best score :  55.43298366099653
Best cover image :  cover_647.jpg


# Selecting Chromosome Preparation

In [24]:
# Two Point Crossover

def two_point_crossover(chromosome1, chromosome2):
    crossover_point1 = random.randint(0, len(chromosome1)-1)
    crossover_point2 = random.randint(0, len(chromosome1)-1)

    while crossover_point1 == crossover_point2:
        crossover_point2 = random.randint(0, len(chromosome1)-1)

    if crossover_point1 > crossover_point2:
        crossover_point1, crossover_point2 = crossover_point2, crossover_point1

    child1 = chromosome1[:crossover_point1] + chromosome2[crossover_point1:crossover_point2] + chromosome1[crossover_point2:]
    child2 = chromosome2[:crossover_point1] + chromosome1[crossover_point1:crossover_point2] + chromosome2[crossover_point2:]

    return child1, child2



# Mutation

def bit_flipMutation(chromosome):
    probability = 1/(len(chromosome))
    temp = list(chromosome)

    for i in range(len(temp)):
        if random.random() < probability:
            if temp[i] == 0:
                temp[i] = 1
            else:
                temp[i] = 0

    chromosome = "".join(str(x) for x in temp)

    return chromosome

In [25]:
# Converting binary to chromosome

def binary_to_chromosome(chromosome):
    # RPosition = int(chromosome[0:2], 2)
    # GPosition = int(chromosome[2:4], 2)
    # BPosition = int(chromosome[4:6], 2)
    partitionValue = int(chromosome[6:9], 2)
    a = int(chromosome[9:25], 2)
    c = int(chromosome[25:41], 2)
    seed = int(chromosome[41:60], 2)
    return [partitionValue, a, c, seed]
    # return [RPosition, GPosition, BPosition, partitionValue, a, c, seed]

In [26]:
# Checking initial seed

def check_chromosome(chromosome):
    if chromosome[3] >= int(cover_imageRow*cover_imageCol*2*3 / (2**chromosome[0])) :
        chromosome[3] = random.randint(0, int(R_secretChannelBinary_1D.shape[0] / (2**chromosome[0])) -1)

    return chromosome

In [27]:
# Roulette wheel for chromosome selection

def roulette_wheel_chromosome_selection(fitness):
       # Computes the total of the population fitness
    population_fitness = 0
    for i in range(len(fitness)):
        population_fitness += fitness[i] 
    
    # print("Population fitness : ",population_fitness)
    
    pick = random.uniform(0, population_fitness)
    # print("Pick : ",pick)

    # Computes for each chromosome the probability 
    current = 0
    for chromosome in range(len(fitness)):
        current += fitness[chromosome]
        if current > pick:
            return chromosome

# Selecting Chromosome

In [28]:
# Tournament Selection

def tournament_selection(fitness, population, tournament_size):
    tournament = []
    for i in range(tournament_size):
        random_index = random.randint(0, len(population)-1)
        tournament.append([fitness[random_index], population[random_index]])
    tournament.sort(reverse=True)
    return tournament[0][1]

In [51]:
# Applying Tournament Selection

os.chdir(COVER_IMAGE_DIRECTORY)

best_score = -10
best_psnr = -10
best_ssim = -10
best_chromosome = -10
best_cover_image = ""

old_fitness = []*len(population)

count_no_change = 0

for iteration in range(5000):  ##########################################1000
    print("Iteration : ",iteration+1)
    

    # Measuring the fitness of each chromosome in the population
    fitness = []*len(population)
    print(len(fitness)," ",len(population))

    starting_index = 0

    if iteration != 0:
        starting_index = len(old_fitness)
        for i in range(len(old_fitness)):
            fitness.append(old_fitness[i])
            print("Chromosome : ",i, "   ",population[i])
            print("Fitness : ",fitness[i])
        old_fitness.clear()

    temp_best_score = best_score
    fitness_list = []*100

    for i in range(starting_index,len(population)):
        
        print("Chromosome : ",i+1,"   ",population[i])
        temp_fitness = 0

        for j in range(1): ######(len(cover_imageList)):
            print("Cover image : ",j)
            R_channelBinary, G_channelBinary, B_channelBinary = imageToNumpyArray(cover_imageList[j])
            R_stegoChannelBinary, G_stegoChannelBinary, B_stegoChannelBinary = create_stego_image(population[i], cover_imageList[j], R_secretChannelBinary_1D, G_secretChannelBinary_1D, B_secretChannelBinary_1D)
            
            mse = mean_squared_error(R_channelBinary, R_stegoChannelBinary) + mean_squared_error(G_channelBinary, G_stegoChannelBinary) + mean_squared_error(B_channelBinary, B_stegoChannelBinary)
            PSNR = calculate_PSNR(mse)
            SSIM = calculate_ssim(R_channelBinary, R_stegoChannelBinary, G_channelBinary, G_stegoChannelBinary, B_channelBinary, B_stegoChannelBinary)
            temp_fitness +=  (0.4* PSNR + 0.6*SSIM) #PSNR
            fitness_list.append(0.4* PSNR + 0.6*SSIM)

            if PSNR > best_psnr:
                best_psnr = PSNR
            if SSIM > best_ssim:
                best_ssim = SSIM
            if (0.4* PSNR + 0.6*SSIM) > best_score:
                best_score = 0.4* PSNR + 0.6*SSIM
                best_chromosome = population[i]
                best_cover_image = cover_imageList[j][1]

            print("Fitness : ", 0.4* PSNR + 0.6*SSIM)
            print("PSNR : ", PSNR)
            print("SSIM : ", SSIM)

        # fitness.append(temp_fitness/len(cover_imageList))
        fitness.append(temp_fitness)

        print("Fitness : ",fitness[i])
        # print()

    if temp_best_score == best_score:
        count_no_change += 1
    else:
        count_no_change = 0

    if count_no_change == 20:
        replace_population = create_population(50)
        
        # replace the worst 50 chromosomes
        for i in range(50):
            # Replace the worst chromosome
            worst_chromosome = fitness_list.index(min(fitness_list))
            population[worst_chromosome] = replace_population[i]

    if count_no_change == 50:
        population = create_population(100)
        count_no_change = 0


        population = create_population(100)

    temp_population = []*len(population)

    # Creating the next generation using crossover and mutation
    k = 0
    while k < 100:  ##########################################1000
        parent_chromosome1 = tournament_selection(fitness, population, 5)
        parent_chromosome2 = tournament_selection(fitness, population, 5)

        parent_chromosome1 = chromosome_to_binary(parent_chromosome1)
        parent_chromosome2 = chromosome_to_binary(parent_chromosome2)

        child1, child2 = two_point_crossover(parent_chromosome1, parent_chromosome2)

        child1 = bit_flipMutation(child1)
        child2 = bit_flipMutation(child2)

        # if(rgb_channelCheck(str(child1)) == False):
        #     child1 = rgb_channelFix(child1)
        # if(rgb_channelCheck(child2) == False):
        #     child2 = rgb_channelFix(child2)

        child1 = binary_to_chromosome(child1)
        child2 = binary_to_chromosome(child2)

        # Checking if seed is within M
        child1 = check_chromosome(child1)
        child2 = check_chromosome(child2)

        # print("Child 1 : ",child1)
        # print("Child 2 : ",child2)

        if child1 not in temp_population:
            temp_population.append(child1)
            k += 1
        if child2 not in temp_population:
            temp_population.append(child2)
            k += 1
  
    population = temp_population

    print("Best score : ",best_score,"  best PSNR : ",best_psnr,"  best SSIM : ",best_ssim,"  Best cover image : ",best_cover_image)
    print()
            


Iteration :  1
0   100
Chromosome :  1     [0, 224, 11298, 78401]
Cover image :  0
Fitness :  22.816936769961288
PSNR :  55.54958675019769
SSIM :  0.9951701164703528
Fitness :  22.816936769961288
Chromosome :  2     [2, 55159, 28147, 60593]
Cover image :  0
Fitness :  22.776017018009938
PSNR :  55.447338747941764
SSIM :  0.9951358647220502
Fitness :  22.776017018009938
Chromosome :  3     [0, 16985, 20603, 25870]
Cover image :  0
Fitness :  22.80050014056013
PSNR :  55.50849786163333
SSIM :  0.9951683265113256
Fitness :  22.80050014056013
Chromosome :  4     [6, 55297, 15660, 748]
Cover image :  0
Fitness :  22.77570050579051
PSNR :  55.44653503636035
SSIM :  0.995144152077279
Fitness :  22.77570050579051
Chromosome :  5     [1, 35277, 20132, 140846]
Cover image :  0
Fitness :  22.77803955997771
PSNR :  55.45238767123688
SSIM :  0.9951408191382621
Fitness :  22.77803955997771
Chromosome :  6     [1, 61329, 52991, 102269]
Cover image :  0
Fitness :  22.772574345517494
PSNR :  55.4387289

KeyboardInterrupt: 

# LSB Embedding Comparison

In [62]:
# Embedding using LSB

def lsb_embed(image, R_secretChannelBinary_1D, G_secretChannelBinary_1D, B_secretChannelBinary_1D):
    # %cd E:\Projects\Thesis\Thesis_Implementation\Dataset\Unsplash_990 
    R_channelBinary, G_channelBinary, B_channelBinary = imageToNumpyArray(image)


    R_init = np.zeros((R_channelBinary.shape[0],R_channelBinary.shape[1]), dtype = int)
    G_init = np.zeros((G_channelBinary.shape[0],G_channelBinary.shape[1]), dtype = int)
    B_init = np.zeros((B_channelBinary.shape[0],B_channelBinary.shape[1]), dtype = int)

    for i in range(R_channelBinary.shape[0]):
        for j in range(R_channelBinary.shape[1]):
            R_init[i][j] = R_channelBinary[i][j]

    for i in range(G_channelBinary.shape[0]):
        for j in range(G_channelBinary.shape[1]):
            G_init[i][j] = G_channelBinary[i][j]

    for i in range(B_channelBinary.shape[0]):
        for j in range(B_channelBinary.shape[1]):
            B_init[i][j] = B_channelBinary[i][j]

    count_secret = 0
    # count_equalR = 0
    for i in range(R_channelBinary.shape[0]):
        for j in range(R_channelBinary.shape[1]):
            if (j+2)%8 == 0 and count_secret < len(R_secretChannelBinary_1D):
                # if(R_init[i][j] == R_secretChannelBinary_1D[count_secret]):
                    # count_equalR += 1
                R_channelBinary[i][j] = R_secretChannelBinary_1D[count_secret]
                count_secret += 1

    # print("Equal R : ",count_equalR)

    count_secret = 0
    for i in range(G_channelBinary.shape[0]):
        for j in range(G_channelBinary.shape[1]):
            if (j+2)%8 == 0 and count_secret < len(G_secretChannelBinary_1D):
                G_channelBinary[i][j] = G_secretChannelBinary_1D[count_secret]
                count_secret += 1

    count_secret = 0
    for i in range(B_channelBinary.shape[0]):
        for j in range(B_channelBinary.shape[1]):
            if (j+2)%8 == 0 and count_secret < len(B_secretChannelBinary_1D):
                B_channelBinary[i][j] = B_secretChannelBinary_1D[count_secret]
                count_secret += 1
            


    mse_R_LSB = np.square(np.subtract(R_channelBinary, R_init)).mean() # mean_squared_error(R_channelBinary, R_stego_using_LSB)
    mse_G_LSB = np.square(np.subtract(G_channelBinary, G_init)).mean() # mean_squared_error(G_channelBinary, G_stego_using_LSB)
    mse_B_LSB = np.square(np.subtract(B_channelBinary, B_init)).mean() # mean_squared_error(B_channelBinary, B_stego_using_LSB)


    mse_LSB = mse_R_LSB + mse_G_LSB + mse_B_LSB
    PSNR_LSB = calculate_PSNR(mse_LSB)
    # print("Mean Square Error : ",mse_LSB)
    print("PSNR : ",PSNR_LSB)

    return PSNR_LSB

# R_stegoChannelBinary, G_stegoChannelBinary, B_stegoChannelBinary = R_stego_using_LSB, G_stego_using_LSB, B_stego_using_LSB

In [63]:
# Creating 10 stego image using LSB

os.chdir(COVER_IMAGE_DIRECTORY)
best_score = 0
for i in range(10):
    print("Cover image : ",cover_imageList[i])
    temp_score = lsb_embed(cover_imageList[i], R_secretChannelBinary_1D, G_secretChannelBinary_1D, B_secretChannelBinary_1D)
    if temp_score > best_score:
        best_score = temp_score
        best_cover_image = cover_imageList[i]

print("Best score : ",best_score,"      Best cover image : ",best_cover_image)

Cover image :  cover_590.jpg
PSNR :  55.306441356774194
Cover image :  cover_402.jpg
PSNR :  55.50636984375815
Cover image :  cover_385.jpg
PSNR :  55.4528347614408
Cover image :  cover_986.jpg
PSNR :  55.83816915730963
Cover image :  cover_095.jpg
PSNR :  55.67961752669824
Cover image :  cover_405.jpg
PSNR :  55.68607613941352
Cover image :  cover_711.jpg
PSNR :  55.62880364730441
Cover image :  cover_597.jpg
PSNR :  55.76323340235141
Cover image :  cover_151.jpg
PSNR :  55.54629625948508
Cover image :  cover_932.jpg
PSNR :  55.57466709181908
Best score :  55.83816915730963       Best cover image :  cover_986.jpg


# Encoding

In [81]:
# Putting together RGB channels to create final stego image

def create_final_stego_image(R_channelBinary, G_channelBinary, B_channelBinary):

    stego_image = np.zeros((R_channelBinary.shape[0], int(R_channelBinary.shape[1]/8), 3), dtype=np.uint8)

    print("Stego Image Shape : ",stego_image.shape)
    print(R_channelBinary.shape)
    
    R_value = ""
    G_value = ""
    B_value = ""
    for i in range(R_channelBinary.shape[0]):
        for j in range(R_channelBinary.shape[1]):
            if j%8 == 0 and j!=0:
                R_value = ""
                G_value = ""
                B_value = ""

             
            R_value += str(R_channelBinary[i][j])
            G_value += str(G_channelBinary[i][j])
            B_value += str(B_channelBinary[i][j])

            if (j+1)%8 == 0:
                stego_image[i][int(j/8)][0] = int(R_value, 2)
                stego_image[i][int(j/8)][1] = int(G_value, 2)
                stego_image[i][int(j/8)][2] = int(B_value, 2)
            
    return stego_image

In [82]:
# CREATE STEGO IMAGE

os.chdir(STEGO_IMAGE_DIRECTORY)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    
print(cover_imageList[1])

warnings.filterwarnings("ignore", category=DeprecationWarning) 
stego_imageArray = create_final_stego_image(R_stegoChannelBinary, G_stegoChannelBinary, B_stegoChannelBinary)
stego_image = Image.fromarray(stego_imageArray)
stego_image.save("stego_image_version11_2.png")

cover_304.jpg
Stego Image Shape :  (256, 256, 3)
(256, 2048)


# Decoding

In [83]:
# Decoding Stego Image to RGB Channel

os.chdir(STEGO_IMAGE_DIRECTORY)

R_decodedChannel, G_decodedChannel, B_decodedChannel = imageToNumpyArray("stego_image_version11_5.png")


# Retrieving chromosome
decoded_chromosome = ""
chromosome_bitCount = 0
for i in range(B_decodedChannel.shape[0]):
    for j in range(B_decodedChannel.shape[1]):
        if (j+3)%8 == 0 and chromosome_bitCount < chromosome_totalBitCount:
            decoded_chromosome += str(B_decodedChannel[i][j])
            chromosome_bitCount += 1

print("Decoded chromosome : ",decoded_chromosome)


# Retrieving data from chromosome
decoded_RPostionStr = ""
decoded_GPostionStr = ""
decoded_BPostionStr = ""
decoded_partitionValueStr = ""
decoded_aStr = ""
decoded_cStr = ""
decoded_seedStr = ""

for i in range(len(decoded_chromosome)):
    if i>=0 and i<=2:
        decoded_partitionValueStr += decoded_chromosome[i]
    elif i>=3 and i<=18:
        decoded_aStr += decoded_chromosome[i]
    elif i>=19 and i<=34:
        decoded_cStr += decoded_chromosome[i]
    elif i>=35 and i<=53:
        decoded_seedStr += decoded_chromosome[i]
    # if i>=0 and i<=1:
    #     decoded_RPostionStr += decoded_chromosome[i]
    # elif i>=2 and i<=3:
    #     decoded_GPostionStr += decoded_chromosome[i]
    # elif i>=4 and i<=5:
    #     decoded_BPostionStr += decoded_chromosome[i]
    # elif i>=6 and i<=8:
    #     decoded_partitionValueStr += decoded_chromosome[i]
    # elif i>=9 and i<=24:
    #     decoded_aStr += decoded_chromosome[i]
    # elif i>=25 and i<=40:
    #     decoded_cStr += decoded_chromosome[i]
    # elif i>=41 and i<=59:
    #     decoded_seedStr += decoded_chromosome[i]

# decoded_RPosition = int(decoded_RPostionStr, 2)
# decoded_GPosition = int(decoded_GPostionStr, 2)
# decoded_BPosition = int(decoded_BPostionStr, 2)
decoded_partitionValue = int(decoded_partitionValueStr, 2)
decoded_a = int(decoded_aStr, 2)
decoded_c = int(decoded_cStr, 2)
decoded_seed = int(decoded_seedStr, 2)

print("R_decodedChannel : ",R_decodedChannel.shape)
# print("Decoded R Position : ",decoded_RPosition)
# print("Decoded G Position : ",decoded_GPosition)
# print("Decoded B Position : ",decoded_BPosition)
print("Decoded Partition Value : ",decoded_partitionValue)
print("Decoded a : ",decoded_a)
print("Decoded c : ",decoded_c)
print("Decoded seed : ",decoded_seed)

Decoded chromosome :  000110101000000000000111010101000111001000000010111011100000
R_decodedChannel :  (256, 2048)
Decoded R Position :  0
Decoded G Position :  1
Decoded B Position :  2
Decoded Partition Value :  5
Decoded a :  14
Decoded c :  43236
Decoded seed :  12000


In [84]:
# Decoding secret image sequence

decoding_sequence = modified_LCG2(decoded_seed, decoded_a, decoded_c, int(cover_imageRow * cover_imageCol*2*3 / (2**decoded_partitionValue))) # (seed, a, c, M):

In [85]:
chromosome = [0, 1 , 2, 5, 14, 43236, 12000]
rearranging_sequence = modified_LCG2(chromosome[6], chromosome[4], chromosome[5], int(cover_imageRow * cover_imageCol*2*3 / (2**chromosome[3])) )

In [86]:
if decoding_sequence == rearranging_sequence:
    print("Same")
else:
    print("Not same")

Same


In [88]:
R_decodedSecretChannelBinary = lastTwoBits_extraction(R_decodedChannel, cover_imageRow, cover_imageCol)
G_decodedSecretChannelBinary = lastTwoBits_extraction(G_decodedChannel, cover_imageRow, cover_imageCol)
B_decodedSecretChannelBinary = lastTwoBits_extraction(B_decodedChannel, cover_imageRow, cover_imageCol)

print(R_decodedSecretChannelBinary.shape)
print(G_decodedSecretChannelBinary.shape)
print(B_decodedSecretChannelBinary.shape)

R_channelBinary_flatten = R_decodedSecretChannelBinary.flatten()
G_channelBinary_flatten = G_decodedSecretChannelBinary.flatten()
B_channelBinary_flatten = B_decodedSecretChannelBinary.flatten()


print("\n",R_channelBinary_flatten.shape)
print(G_channelBinary_flatten.shape)
print(B_channelBinary_flatten.shape)

concatenated_array = np.concatenate((R_channelBinary_flatten, G_channelBinary_flatten, B_channelBinary_flatten), axis=0)
print(concatenated_array.shape)
    

(65536, 2)
(65536, 2)
(65536, 2)

 (131072,)
(131072,)
(131072,)
(393216,)


In [89]:
decoded_secretBits = np.zeros(secret_imageRow*secret_imageCol*8*3, dtype=int)

secret_channelIndex = 0

for i in range(len(decoding_sequence)):
    for j in range((2**decoded_partitionValue)*decoding_sequence[i], (2**decoded_partitionValue)*(decoding_sequence[i]+1)):
        if secret_channelIndex >= 64*128*24:
            break
        decoded_secretBits[secret_channelIndex] = concatenated_array[j]
        secret_channelIndex += 1

print(secret_channelIndex)
print(decoded_secretBits.shape)

196608
(196608,)


In [90]:
R_decodedSecretChannelBinary = decoded_secretBits[:secret_imageRow*secret_imageCol*8]
G_decodedSecretChannelBinary = decoded_secretBits[secret_imageRow*secret_imageCol*8 : secret_imageRow*secret_imageCol*8*2]
B_decodedSecretChannelBinary = decoded_secretBits[secret_imageRow*secret_imageCol*8*2 : secret_imageRow*secret_imageCol*8*3]

print(R_decodedSecretChannelBinary.shape)
print(G_decodedSecretChannelBinary.shape)
print(B_decodedSecretChannelBinary.shape)

R_decodedSecretChannelBinary = R_decodedSecretChannelBinary.reshape(secret_imageRow, secret_imageCol*8)
G_decodedSecretChannelBinary = G_decodedSecretChannelBinary.reshape(secret_imageRow, secret_imageCol*8)
B_decodedSecretChannelBinary = B_decodedSecretChannelBinary.reshape(secret_imageRow, secret_imageCol*8)

print(R_decodedSecretChannelBinary.shape)
print(G_decodedSecretChannelBinary.shape)
print(B_decodedSecretChannelBinary.shape)

(65536,)
(65536,)
(65536,)
(64, 1024)
(64, 1024)
(64, 1024)


In [91]:
def create_decoded_secret_image(R_channelBinary, G_channelBinary, B_channelBinary):
    
    print("R_channelBinary.shape[0] : ",R_channelBinary.shape[0])
    print("R_channelBinary.shape[1] : ",R_channelBinary.shape[1])

    decoded_secretImage = np.zeros((R_channelBinary.shape[0], int(R_channelBinary.shape[1]/8), 3), dtype=np.uint8)


    R_value = ""
    G_value = ""
    B_value = ""
    for i in range(R_channelBinary.shape[0]):
        for j in range(R_channelBinary.shape[1]):
            if j%8 == 0 and j!=0:
                R_value = ""
                G_value = ""
                B_value = ""

             
            R_value += str(R_channelBinary[i][j])
            G_value += str(G_channelBinary[i][j])
            B_value += str(B_channelBinary[i][j])

            if (j+1)%8 == 0:
                decoded_secretImage[i][int(j/8)][0] = int(R_value, 2)
                decoded_secretImage[i][int(j/8)][1] = int(G_value, 2)
                decoded_secretImage[i][int(j/8)][2] = int(B_value, 2)

    # for i in range(1):
    #     for j in range(5):
    #         print(i,",",j,"  ",decoded_secretImage[i][j][0],"   ",format(decoded_secretImage[i][j][0], '08b'))         
            
    return decoded_secretImage

In [92]:
# Create decoded secret image

os.chdir(DECODED_IMAGE_DIRECTORY)

warnings.filterwarnings("ignore", category=DeprecationWarning) 
decoded_secretImageArray = create_decoded_secret_image(R_decodedSecretChannelBinary, G_decodedSecretChannelBinary, B_decodedSecretChannelBinary)

decoded_secretImage = Image.fromarray(decoded_secretImageArray)
decoded_secretImage.save("decoded_secretImage_version11_3.png")

R_channelBinary.shape[0] :  64
R_channelBinary.shape[1] :  1024
